# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [ ]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [ ]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 10, 5, 1),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.conv2 = nn.Sequential(
            nn.Conv2d(10, 20, 5, 1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25))
        self.fc1 = nn.Sequential(
            nn.Linear(320, 50),
            nn.ReLU(),
            nn.Dropout(0.5))
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(-1, 320)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

model = SimpleNet()

# TODO:define loss function and optimiter
criterion = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Next, we can start to train and evaluate!

In [ ]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        if epoch >= 5:
            optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
        optimizer.zero_grad()
        output = model(images)
        lsm = nn.LogSoftmax(1)
        output = lsm(output)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()     
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    model.eval()
    correct = 0
    for images, labels in tqdm(train_loader):
        output = model(images)
        lsm = nn.LogSoftmax(1)
        output = lsm(output)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(labels.view_as(pred)).sum().item()
    train_accuracy = correct / len(train_loader.dataset)
    print('Training accuracy: %0.2f%%' % (train_accuracy*100))
    correct = 0
    for images, labels in tqdm(test_loader):
        output = model(images)
        lsm = nn.LogSoftmax(1)
        output = lsm(output)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(labels.view_as(pred)).sum().item()
    test_accuracy = correct/len(test_loader.dataset)
    print('Testing accuracy: %0.2f%%' % (test_accuracy * 100))

#### Q5:
Please print the training and testing accuracy.